In [21]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import datetime

In [181]:
def get_prices(date):
    A1_OA_Da = 16                     # Day Align
    A1_OA_Ta = "America/Mexico_City"  # Time Align
    A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
    A1_OA_At = "practice"             # Tipo de cuenta
    A1_OA_In = "USD_MXN"              # Instrumento
    A1_OA_Gn = "M1"                   # Granularidad de velas
    A1_OA_Ak = '2' + '6b62ddbe404c61a1cc9da0ed8395945-52d44ef76c42b62460581783bba6c8e' + 'a'
    ##obtener el spot
    a=datetime.datetime.strptime(date,"%m/%d/%Y %H:%M")
    today = a
    DD = datetime.timedelta(minutes=11)
    earlier = today - DD
    later = today + DD
    F1=later.strftime("%Y-%m-%dT%H:%M:%SZ")
    F2=earlier.strftime("%Y-%m-%dT%H:%M:%SZ")
    api = API(access_token=A1_OA_Ak)
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
              "alignmentTimezone": A1_OA_Ta, "from": F2, "to": F1}
    A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    a=list(map(lambda i:[A1_Hist['candles'][i]['time'],float(A1_Hist['candles'][i]['mid']['o']),float(A1_Hist['candles'][i]['mid']['h']),
                          float(A1_Hist['candles'][i]['mid']['l']),float(A1_Hist['candles'][i]['mid']['c'])],range(len(A1_Hist['candles']))))
    pd_hist = pd.DataFrame(a,columns=['TimeStamp','Open','High','Low','Close'])
    pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
    pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
    
    return pd_hist 

In [182]:
data=pd.read_csv('data.csv')

In [183]:
a=data.loc[[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus>=data.Previous)]]
b=data.loc[[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus<data.Previous)]]
c=data.loc[[a and b for a,b in zip(data.Actual<data.Consensus,data.Consensus>=data.Previous)]]
d=data.loc[[a and b for a,b in zip(data.Actual<=data.Consensus,data.Consensus<data.Previous)]]
data['a']=[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus>=data.Previous)]
data['b']=[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus<data.Previous)]
data['c']=[a and b for a,b in zip(data.Actual<data.Consensus,data.Consensus>=data.Previous)]
data['d']=[a and b for a,b in zip(data.Actual<=data.Consensus,data.Consensus<data.Previous)]


In [184]:
week_days=list(map(lambda x:datetime.datetime.weekday(datetime.datetime.strptime(data.DateTime[x],"%m/%d/%Y %H:%M")),range(len(data))))

In [185]:
temp=list(map(lambda x:get_prices(data.DateTime[x]),range(len(data))))

In [298]:
c1_direction=list(map(lambda x:temp[x].loc[21].Close-temp[x].loc[11].Open,range(len(temp))))

In [299]:
c2_vol=list(map(lambda x:max(temp[x].High)-min(temp[x].Low),range(len(temp))))

In [300]:
c3_drawDown=list(map(lambda x:temp[x].loc[0].Open-temp[x].loc[21].Low,range(len(temp))))

In [301]:
c4_drawUp=list(map(lambda x:temp[x].loc[0].Open-temp[x].loc[21].High,range(len(temp))))

In [302]:
data['c1_direction']=c1_direction
data['c2_vol']=c2_vol
data['c3_drawDown']=c3_drawDown
data['c4_drawUp']=c4_drawUp


In [304]:
po=[sum(data['a'])/len(data),
sum(data['b'])/len(data),
sum(data['c'])/len(data),
sum(data['d'])/len(data)]


In [305]:
tp=np.array([sum(data.loc[a.index]['c1_direction']>0)/sum(data['a']),
sum(data.loc[b.index]['c1_direction']>0)/sum(data['b']),
sum(data.loc[c.index]['c1_direction']>0)/sum(data['c']),
sum(data.loc[d.index]['c1_direction']>0)/sum(data['d'])])

In [306]:
tn=1-tp

In [307]:
results=np.transpose(pd.DataFrame([tp,tn,po],columns=['a','b','c','d'],index=['direccion positiva','direccion negativa','porcentaje ocurrencia']))

In [326]:
results

,direccion positiva,direccion negativa,porcentaje ocurrencia
a,0.428571,0.571429,0.205882
b,0.000000,1.000000,0.029412
c,0.400000,0.600000,0.588235
d,1.000000,0.000000,0.176471


In [317]:
total=10000
trade=[]
for i in range(len(temp)):
    if (results.loc[data.iloc[i,7:11]]['direccion positiva']==1)[0]:

        dolares_totales=np.trunc(total/temp[i].loc[0].Open)
        monto_compra=dolares_totales*temp[i].loc[11].Open
        total-=monto_compra
        monto_venta=dolares_totales*temp[i].loc[21].Close
        pL=monto_venta-monto_compra
        total+=monto_venta
        trade.append([dolares_totales,pL,total])
    else:
        trade.append([0,0,total])


In [323]:
trades=pd.DataFrame(trade,columns=['total quantity','P&L','Total'])
trades

,total quantity,P&L,Total
0,577.0,4.65062,10004.65062
1,0.0,0.00000,10004.65062
2,0.0,0.00000,10004.65062
3,573.0,18.03804,10022.68866
4,0.0,0.00000,10022.68866
5,0.0,0.00000,10022.68866
6,0.0,0.00000,10022.68866
7,0.0,0.00000,10022.68866
8,0.0,0.00000,10022.68866
9,0.0,0.00000,10022.68866


In [324]:
data=data.join(trades)

In [325]:
data

,DateTime,Name,Country,Volatility,Actual,Previous,Consensus,a,b,c,d,c1_direction,c2_vol,c3_drawDown,c4_drawUp,total quantity,P&L,Total
0,01/04/2016 15:00,Construction Spending (MoM),United States,3,-0.4,1.0,0.6,False,False,False,True,0.00806,0.04112,-0.01069,-0.01420,577.0,4.65062,10004.65062
1,02/01/2016 15:00,Construction Spending (MoM),United States,3,0.1,-0.4,0.6,False,False,True,False,0.00123,0.07737,-0.00106,-0.01585,0.0,0.00000,10004.65062
2,03/01/2016 15:00,Construction Spending (MoM),United States,3,1.5,0.1,0.4,True,False,False,False,0.00240,0.04593,0.01820,0.00841,0.0,0.00000,10004.65062
3,04/01/2016 14:00,Construction Spending (MoM),United States,3,-0.5,1.5,0.1,False,False,False,True,0.03148,0.05745,-0.03458,-0.03996,573.0,18.03804,10022.68866
4,05/02/2016 14:00,Construction Spending (MoM),United States,3,0.3,-0.5,0.5,False,False,True,False,0.04768,0.05671,-0.02342,-0.04886,0.0,0.00000,10022.68866
5,06/01/2016 14:00,Construction Spending (MoM),United States,3,-1.8,0.3,0.6,False,False,True,False,-0.02625,0.06598,0.03456,0.01816,0.0,0.00000,10022.68866
6,07/01/2016 14:00,Construction Spending (MoM),United States,3,-0.8,-1.8,0.6,False,False,True,False,0.04604,0.06099,-0.04246,-0.05259,0.0,0.00000,10022.68866
7,08/01/2016 14:00,Construction Spending (MoM),United States,3,-0.6,-0.8,0.5,False,False,True,False,0.01524,0.03755,0.01040,0.00482,0.0,0.00000,10022.68866
8,09/01/2016 14:00,Construction Spending (MoM),United States,3,0.0,-0.6,0.5,False,False,True,False,-0.04484,0.09527,0.05431,0.04266,0.0,0.00000,10022.68866
9,10/03/2016 14:00,Construction Spending (MoM),United States,3,-0.7,0.0,0.2,False,False,True,False,0.01884,0.03898,-0.01702,-0.02484,0.0,0.00000,10022.68866
